# Building MOP-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [1]:
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import warnings
warnings.filterwarnings('ignore')

First we get the MOP-terphenyl monomer from a smiles string.

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3cccc(c3)[C@H](N)C))cc1', smiles = True, name="MOP")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.212  -0.58   -0.0471]), 1 bonds, id: 139962245819184>
26 <H pos=([-0.0898 -0.3998  0.0731]), 1 bonds, id: 139962245820384>
27 <H pos=([-0.05   -0.163   0.1233]), 1 bonds, id: 139962245821248>
28 <H pos=([0.0677 0.2914 0.0327]), 1 bonds, id: 139962245822112>
29 <H pos=([-0.1634  0.6468  0.1882]), 1 bonds, id: 139962245956368>
30 <H pos=([-0.2546  0.5109  0.2639]), 1 bonds, id: 139962245955840>
31 <H pos=([-0.2903  0.5616  0.0926]), 1 bonds, id: 139962245957664>
32 <H pos=([-0.3495  0.3241  0.1313]), 1 bonds, id: 139962245821680>
33 <H pos=([-0.3838  0.0902  0.0722]), 1 bonds, id: 139962245821344>
34 <H pos=([-0.0312 -0.0175 -0.2343]), 1 bonds, id: 139962245823120>
35 <H pos=([ 0.1606 -0.1453 -0.3215]), 1 bonds, id: 139962245824176>
36 <H pos=([ 0.3677 -0.1653 -0.1902]), 1 bonds, id: 139962245823888>
37 <H pos=([0.1908 0.073  0.123 ]), 1 bonds, id: 139962245957856>
38 <H pos=([ 0.4982 -0.1177 -0.0019]), 1 bonds, id: 139962245955696>
39 <H pos=([ 0.3596 -0.2263  0.1593]), 1

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks. Using the object from `py3Dmol` we can coloro the atoms to identify the indices needed to make substiutions when building the polymer.

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [7]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 40],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')

In [8]:
chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
for p in chain.particles():
    print(p)

<C pos=([-0.2723 -0.5048 -0.0963]), 3 bonds, id: 139962205062432>
<O pos=([-0.3612 -0.5415 -0.1777]), 1 bonds, id: 139962205062624>
<C pos=([-0.2496 -0.3619 -0.0677]), 3 bonds, id: 139962205062768>
<C pos=([-0.1498 -0.3245  0.0235]), 3 bonds, id: 139962205062912>
<C pos=([-0.1268 -0.1896  0.0517]), 3 bonds, id: 139962205063056>
<C pos=([-0.2026 -0.0898 -0.0116]), 3 bonds, id: 139962205063200>
<C pos=([-0.1778  0.0535  0.0173]), 3 bonds, id: 139962205063344>
<C pos=([-0.049   0.1115  0.0052]), 3 bonds, id: 139962205063488>
<C pos=([-0.0303  0.246   0.0417]), 3 bonds, id: 139962205063632>
<C pos=([-0.137   0.3239  0.0872]), 3 bonds, id: 139962205063776>
<O pos=([-0.1119  0.4577  0.1214]), 2 bonds, id: 139962205063920>
<C pos=([-0.2108  0.5484  0.169 ]), 4 bonds, id: 139962205064064>
<C pos=([-0.2649  0.2665  0.0966]), 3 bonds, id: 139962205064208>
<C pos=([-0.2847  0.1328  0.062 ]), 3 bonds, id: 139962205064352>
<C pos=([ 0.0666  0.0359 -0.0487]), 3 bonds, id: 139962205064496>
<C pos=([ 

In [10]:
# no need for rotation of peptide bonds
#for bond in chain.bonds():
#    if bond[0].name == "N":
#        if bond[1].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)
#            
#    if bond[1].name == "N":
#        if bond[0].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)       

In [11]:
chain.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [12]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 139962204972720>, <Compound 44 particles, 46 bonds, non-periodic, id: 139962204963648>, <Compound 44 particles, 46 bonds, non-periodic, id: 139962245970480>, <Compound 44 particles, 46 bonds, non-periodic, id: 139962246203328>, <Compound 44 particles, 46 bonds, non-periodic, id: 139962205322496>, <Compound 44 particles, 46 bonds, non-periodic, id: 139962205726720>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962204972720>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962204963648>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962245970480>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962246203328>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962205322496>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 139962205726720>), ('Compound', [<Compound 5 particles,

We save these as a pdb file and provide the names of the residues to include in the file.

In [13]:
chain.save("mop_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [14]:
! gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 34.4435 nm^3, corresponds to roughly 15400 electrons
No velocities found

Back Off! I just backed up mop_hexamer_mbuild.gro to ./#mop_hexamer_mbuild.gro.3#

GROMACS reminds you: "Science progresses best when observations force us to alter our preconceptions." (Vera Rubin)



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [15]:
! obabel -ipdb mop_hexamer_mbuild.pdb -omol mop_hexamer_mbuild.mol -O mop_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


# Building arbitrary n-mers of MOP monomer

Now we put it all together to generate a tetramer, hexamer and octamer PMP terphenyl foldamer:

In [16]:
n_residues = [2,3,4,6,8,10]
residue_labels = ["DIM", "TRI","TET", "HEX", "OCT", "DEC"]
file_names = ["mop_dimer_mbuild", "mop_trimer_mbuild", "mop_tetramer_mbuild", "mop_hexamer_mbuild", "mop_octamer_mbuild", "mop_decamer_mbuild"]
for i in range(len(n_residues)):
    # Generate polymer
    chain = Polymer()
    chain.add_monomer(compound=comp,
                      indices=[25, 40],
                      separation=.15,
                      replace=True,
                      # orientation = [[0,-1,0],[1,0,0]]
                     )
    chain.add_end_groups(compound = cap_o,
                         index = -1,
                         separation=0.15,
                         label="head",
                         duplicate = False
                        )

    chain.add_end_groups(compound = cap_n,
                         index = -1,
                         separation=0.15,
                         label="tail",
                         duplicate = False
                        )

    chain.build(n=n_residues[i], sequence='A')

    # Rotate peptide bond
    #for bond in chain.bonds():
    #    if bond[0].name == "N":
    #        if bond[1].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)

    #    if bond[1].name == "N":
    #        if bond[0].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)  


    # Relabel chains
    for label in chain.labels["monomer"]:
        label.name = residue_labels[i]
        print(label)
    for label in chain.labels["Compound"]:
        label.name = "CAP"
        print(label)
            
            
    chain.save(file_names[i] + ".pdb", overwrite=True, residues=[residue_labels[i], "CAP"])
    
    subprocess.run(["gmx", "editconf", "-f", file_names[i] + ".pdb", "-o", file_names[i]+ ".gro"])
    subprocess.run(["obabel", "-ipdb", file_names[i] + ".pdb", "-omol", file_names[i] + ".mol",  "-O", file_names[i] + ".mol"])
    subprocess.run(["renumber_pdb_atoms", "-f", file_names[i] + ".pdb", "-o", file_names[i] + "_renum.pdb"])

<DIM 44 particles, 46 bonds, non-periodic, id: 139962185906688>
<DIM 44 particles, 46 bonds, non-periodic, id: 139962185904192>
<CAP 5 particles, 4 bonds, non-periodic, id: 139962185175520>
<CAP 16 particles, 15 bonds, non-periodic, id: 139962185892576>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 109 atoms
Volume: 8.96552 nm^3, corresponds to roughly 4000 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_dimer_mbuild.pdb -o mop_dimer_mbuild.gro


Back Off! I just backed up mop_dimer_mbuild.gro to ./#mop_dimer_mbuild.gro.2#

GROMACS reminds you: "The future still looks good, and you've got time to rectify all the things that you should" (G. Harrison)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<TRI 44 particles, 46 bonds, non-periodic, id: 139962203687808>
<TRI 44 particles, 46 bonds, non-periodic, id: 139962203689296>
<TRI 44 particles, 46 bonds, non-periodic, id: 139962927955248>
<CAP 5 particles, 4 bonds, non-periodic, id: 139962185141984>
<CAP 16 particles, 15 bonds, non-periodic, id: 139962203697312>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 153 atoms
Volume: 12.894 nm^3, corresponds to roughly 5800 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_trimer_mbuild.pdb -o mop_trimer_mbuild.gro


Back Off! I just backed up mop_trimer_mbuild.gro to ./#mop_trimer_mbuild.gro.2#

GROMACS reminds you: "And after some more talk we agreed that the wisdom of rats had been grossly overrated, being in fact no greater than that of men" (Joseph Conrad)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<TET 44 particles, 46 bonds, non-periodic, id: 139962928126688>
<TET 44 particles, 46 bonds, non-periodic, id: 139962928124192>
<TET 44 particles, 46 bonds, non-periodic, id: 139962184951808>
<TET 44 particles, 46 bonds, non-periodic, id: 139962928249632>
<CAP 5 particles, 4 bonds, non-periodic, id: 139962185473456>
<CAP 16 particles, 15 bonds, non-periodic, id: 139962185466304>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 197 atoms
Volume: 16.1929 nm^3, corresponds to roughly 7200 electrons
No velocities found


...

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_octamer_mbuild.pdb -o mop_octamer_mbuild.gro


Back Off! I just backed up mop_octamer_mbuild.gro to ./#mop_octamer_mbuild.gro.2#

GROMACS reminds you: "Act like Prometheus would" (Gogol Bordello)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<DEC 44 particles, 46 bonds, non-periodic, id: 139962924394976>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962924396944>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962922514704>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962922630208>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962922794768>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962922973840>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962923187120>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962923006224>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962923416160>
<DEC 44 particles, 46 bonds, non-periodic, id: 139962921565776>
<CAP 5 particles, 4 bonds, non-periodic, id: 139962924285376>
<CAP 16 particles, 15 bonds, non-periodic, id: 139962924283792>


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/terphenyl_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_decamer_mbuild.pdb -o mop_decamer_mbuild.gro


Back Off! I just backed up mop_decamer_mbuild.gro to ./#mop_decamer_mbuild.gro.2#

GROMACS reminds you: "We must have perseverance and above all confidence in ourselves. We must believe that we are gifted for something and that this thing must be attained." (Marie Curie)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 461 atoms
Volume: 82.9883 nm^3, corresponds to roughly 37300 electrons
No velocities found


*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
